In [1]:
import numpy as np
import generate_HNF
import spglib
from copy import deepcopy
import permutation
import generate_superlattice

rutile = ([(4, 0, 0),

           (0, 4, 0),

           (0, 0, 3)],

          [(0, 0, 0),

           (0.5, 0.5, 0.5),

           (0.3, 0.3, 0.0),

           (0.7, 0.7, 0.0),

           (0.2, 0.8, 0.5),

           (0.8, 0.2, 0.5)],

          [14, 14, 8, 8, 8, 8])
o_sublattice = ([(4, 0, 0),

                 (0, 4, 0),

                 (0, 0, 3)],

                [(0.3, 0.3, 0.0),

                 (0.7, 0.7, 0.0),

                 (0.2, 0.8, 0.5),

                 (0.8, 0.2, 0.5)],

                [8, 8, 8, 8])
parent_sym = spglib.get_symmetry(rutile)
HNF_list = generate_HNF.generate_all_superlattices(3)
reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(
    HNF_list, parent_sym["rotations"])
reduced_HNF

[array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 3]]), array([[1, 0, 0],
        [0, 1, 0],
        [0, 1, 3]]), array([[1, 0, 0],
        [0, 1, 0],
        [1, 1, 3]]), array([[1, 0, 0],
        [0, 3, 0],
        [0, 0, 1]]), array([[1, 0, 0],
        [1, 3, 0],
        [0, 0, 1]])]

In [2]:
# reduced_HNF[1]について
H = reduced_HNF[2]
H_inv = np.linalg.inv(H)

def is_unimodular(M: np.ndarray) -> bool:

    if np.abs(np.around(np.linalg.det(M))) == 1:

        return True

    else:

        return False
def cast_integer_matrix(arr: np.ndarray) -> np.ndarray:

    arr_int = np.around(arr).astype(np.int)

    return arr_int
def is_integer_matrix(matrix):
    a = True
    for i in range(0, 3):
        for j in range(0, 3):
            if not matrix[i][j].is_integer():
                a = False
                break
    return a

for i in range(len(parent_sym["rotations"])):
    check = H_inv.dot(parent_sym["rotations"][i].dot(H))
    if is_integer_matrix(check):
        print(check)

print("H_inv", H_inv)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
H_inv [[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [-0.33333333 -0.33333333  0.33333333]]


この方法で得られた部分点群と並進操作を組み合わせてうまくいくか

In [3]:
class Superlattice_spacegroup():
    def __init__(self, parent_sym, HNF):
        self.parent_sym = parent_sym
        self.HNF = HNF
    def print_HNF(self):
        print(self.HNF)

In [4]:
a = Superlattice_spacegroup(parent_sym, reduced_HNF[0])

In [5]:
b = a.print_HNF()

[[1 0 0]
 [0 1 0]
 [0 0 3]]


In [6]:
def get_superlattice_symmetry(parent_sym, HNF):
    H = HNF
    H_inv = np.linalg.inv(H)
    Subspacegroup = dict()
    Subspacegroup["rotations"] = list()
    Subspacegroup["translations"] = list()
    for i in range(len(parent_sym["rotations"])):
        check = H_inv.dot(parent_sym["rotations"][i].dot(H))
        if is_integer_matrix(check):
            t = parent_sym["translations"][i]
            t_neo = np.dot(H_inv, t)
            Subspacegroup["rotations"].append(check)
            Subspacegroup["translations"].append(t_neo)
    return Subspacegroup

In [7]:
hu  = get_superlattice_symmetry(parent_sym, reduced_HNF[0])
hu

{'rotations': [array([[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]), array([[-1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]]), array([[ 0., -1.,  0.],
         [ 1.,  0.,  0.],
         [ 0.,  0.,  1.]]), array([[ 0.,  1.,  0.],
         [-1.,  0.,  0.],
         [ 0.,  0., -1.]]), array([[-1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0.,  1.]]), array([[ 1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0., -1.]]), array([[ 0.,  1.,  0.],
         [-1.,  0.,  0.],
         [ 0.,  0.,  1.]]), array([[ 0., -1.,  0.],
         [ 1.,  0.,  0.],
         [ 0.,  0., -1.]]), array([[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]]), array([[-1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  1.]]), array([[ 0., -1.,  0.],
         [-1.,  0.,  0.],
         [ 0.,  0., -1.]]), array([[0., 1., 0.],
         [1., 0., 0.],
         [0., 0., 1.]]), array([[-1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0

新座標系での表現に直したい,
そんで新しく生じる並進操作と組み合わせる

In [10]:
def shin_get_permutation(o_super_sublattice,parent_sym, HNF):
    dic_zahyou = dict()                        #分率座標の行列表示
    for i in range(len(o_super_sublattice[1])):
        dic_zahyou[i] = np.asarray(o_super_sublattice[1][i])
    trans = permutation.gene_trans(HNF)
    trans_tikans = permutation.get_trans_perms(dic_zahyou, trans)
    super_point_group = get_superlattice_symmetry(parent_sym, HNF)
    def generate_super_perm(dic_zahyou, super_point_group):##各座標をnumpyに変換して、gene_perを使う
        jun = dict()
        for i in range(len(super_point_group["translations"])):
            jun[i] = permutation.gene_per(super_point_group["rotations"][i], super_point_group["translations"][i], dic_zahyou)
        return jun
    
    rot_permutations = generate_super_perm(dic_zahyou, super_point_group)
    print(rot_permutations, "rotperm")
    print(trans_tikans, "trans_tikans")
    goal_perms = permutation.generate_abs_permuatation(rot_permutations, trans_tikans)
    
    return goal_perms

In [11]:
o_super_sublattice = generate_superlattice.get_o_superlattice(rutile, reduced_HNF[0], 3)
unko = shin_get_permutation(o_super_sublattice, parent_sym,  reduced_HNF[0])

{0: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11}, 1: {0: 1, 1: 0, 2: 11, 3: 10, 4: 9, 5: 8, 6: 7, 7: 6, 8: 5, 9: 4, 10: 3, 11: 2}, 2: {0: 2, 1: 3, 4: 6, 5: 7, 6: 9, 7: 8}, 3: {0: 3, 1: 2, 6: 8, 7: 9, 8: 7, 9: 6}, 4: {0: 1, 1: 0, 2: 3, 3: 2, 4: 5, 5: 4, 6: 7, 7: 6, 8: 9, 9: 8, 10: 11, 11: 10}, 5: {0: 0, 1: 1, 2: 10, 3: 11, 4: 8, 5: 9, 6: 6, 7: 7, 8: 4, 9: 5, 10: 2, 11: 3}, 6: {0: 3, 1: 2, 4: 7, 5: 6, 6: 8, 7: 9}, 7: {0: 2, 1: 3, 6: 9, 7: 8, 8: 6, 9: 7}, 8: {0: 3, 1: 2, 6: 9, 7: 8, 8: 7, 9: 6}, 9: {0: 2, 1: 3, 4: 6, 5: 7, 6: 8, 7: 9}, 10: {0: 1, 1: 0, 2: 10, 3: 11, 4: 9, 5: 8, 6: 6, 7: 7, 8: 5, 9: 4, 10: 2, 11: 3}, 11: {0: 0, 1: 1, 2: 3, 3: 2, 4: 4, 5: 5, 6: 7, 7: 6, 8: 8, 9: 9, 10: 11, 11: 10}, 12: {0: 2, 1: 3, 6: 8, 7: 9, 8: 6, 9: 7}, 13: {0: 3, 1: 2, 4: 7, 5: 6, 6: 9, 7: 8}, 14: {0: 0, 1: 1, 2: 11, 3: 10, 4: 8, 5: 9, 6: 7, 7: 6, 8: 4, 9: 5, 10: 3, 11: 2}, 15: {0: 1, 1: 0, 2: 2, 3: 3, 4: 5, 5: 4, 6: 6, 7: 7, 8: 9, 9: 8, 10: 10, 11: 11}} rotperm
{0: {0: 0

KeyError: 2

In [43]:
unko

[{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11},
 {0: 4, 1: 5, 2: 6, 3: 7, 4: 8, 5: 9, 6: 10, 7: 11, 8: 0, 9: 1, 10: 2, 11: 3},
 {0: 8, 1: 9, 2: 10, 3: 11, 4: 0, 5: 1, 6: 2, 7: 3, 8: 4, 9: 5, 10: 6, 11: 7},
 {0: 11, 1: 10, 2: 9, 3: 8, 4: 7, 5: 6, 6: 5, 7: 4, 8: 3, 9: 2, 10: 1, 11: 0},
 {0: 3, 1: 2, 2: 1, 3: 0, 4: 11, 5: 10, 6: 9, 7: 8, 8: 7, 9: 6, 10: 5, 11: 4},
 {0: 7, 1: 6, 2: 5, 3: 4, 4: 3, 5: 2, 6: 1, 7: 0, 8: 11, 9: 10, 10: 9, 11: 8},
 {0: 9, 1: 8, 2: 11, 3: 10, 4: 5, 5: 4, 6: 7, 7: 6, 8: 1, 9: 0, 10: 3, 11: 2},
 {0: 1, 1: 0, 2: 3, 3: 2, 4: 9, 5: 8, 6: 11, 7: 10, 8: 5, 9: 4, 10: 7, 11: 6},
 {0: 5, 1: 4, 2: 7, 3: 6, 4: 1, 5: 0, 6: 3, 7: 2, 8: 9, 9: 8, 10: 11, 11: 10},
 {0: 2, 1: 3, 2: 0, 3: 1, 4: 6, 5: 7, 6: 4, 7: 5, 8: 10, 9: 11, 10: 8, 11: 9},
 {0: 6, 1: 7, 2: 4, 3: 5, 4: 10, 5: 11, 6: 8, 7: 9, 8: 2, 9: 3, 10: 0, 11: 1},
 {0: 10, 1: 11, 2: 8, 3: 9, 4: 2, 5: 3, 6: 0, 7: 1, 8: 6, 9: 7, 10: 4, 11: 5}]

In [21]:
def sucell_unique(rutile, HNF, index):
    parent_lattice = generate_superlattice.get_superlattice(rutile, HNF, index)
    o_sublattice = generate_superlattice.get_o_superlattice(rutile, HNF, index)
    goalen = shin_get_permutation(o_sublattice, parent_sym, HNF)
    ds_list = list()
    for oxy_number in range(index*4):
        can = permutation.make_candidate(o_sublattice, oxy_number)
        auncko = permutation.unique(goalen, can)
        ds_list.extend(auncko)
    return ds_list

In [29]:
HNF_list = generate_HNF.generate_all_superlattices(3)
reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, parent_sym["rotations"])
ds = sucell_unique(rutile, reduced_HNF[0], 3)
print(len(ds))

0 {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11}
{0: 1, 1: 0, 2: 11, 3: 10, 4: 9, 5: 8, 6: 7, 7: 6, 8: 5, 9: 4, 10: 3, 11: 2}
1 {0: 4, 1: 5, 2: 6, 3: 7, 4: 8, 5: 9, 6: 10, 7: 11, 8: 0, 9: 1, 10: 2, 11: 3}
{0: 5, 1: 4, 2: 3, 3: 2, 4: 1, 5: 0, 6: 11, 7: 10, 8: 9, 9: 8, 10: 7, 11: 6}
2 {0: 8, 1: 9, 2: 10, 3: 11, 4: 0, 5: 1, 6: 2, 7: 3, 8: 4, 9: 5, 10: 6, 11: 7}
{0: 9, 1: 8, 2: 7, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1, 9: 0, 10: 11, 11: 10}


KeyError: 2

In [14]:
def sucell_unique(rutile, HNF, index):
    parent_lattice = generate_superlattice.get_superlattice(rutile, HNF, index)
    o_sublattice = generate_superlattice.get_o_superlattice(rutile, HNF, index)
    goalen = shin_get_permutation(o_sublattice, parent_sym, HNF)
    ds_list = list()
    for oxy_number in range(index*4):
        can = permutation.make_candidate(o_sublattice, oxy_number)
        auncko = permutation.unique(goalen, can)
        ds_list.extend(auncko)
    return ds_list

def shin_get_permutation(o_super_sublattice,parent_sym, HNF):
    def kumiawase(rot_tikan, trans_tikan):
        """
        回転操作と並進操作の置換を組合す

        parameters

        retruns

        """
        hosii = dict()
        for hayashi in range(len(rot_tikan)):
            hosii[hayashi] = trans_tikan[rot_tikan[hayashi]]
        return hosii

    def generate_abs_permuatation(parent_lattice_jun, trans_perms):
        """
        回転操作と並進操作の置換を組合せた究極の辞書を作成

        parameters

        retruns

        """
        tikan_list = list()
        for tera in parent_lattice_jun:
            for hoge in trans_perms:
                tikan = kumiawase(parent_lattice_jun[tera], trans_perms[hoge])
                tikan_list.append(tikan)
        return tikan_list
    
    dic_zahyou = dict()                        #分率座標の行列表示
    for i in range(len(o_super_sublattice[1])):
        dic_zahyou[i] = np.asarray(o_super_sublattice[1][i])
    trans = permutation.gene_trans(HNF)
    trans_tikans = permutation.get_trans_perms(dic_zahyou, trans)
    super_point_group = get_superlattice_symmetry(parent_sym, HNF)
    def generate_super_perm(dic_zahyou, super_point_group):##各座標をnumpyに変換して、gene_perを使う
        jun = dict()
        for i in range(len(super_point_group["translations"])):
            jun[i] = permutation.gene_per(super_point_group["rotations"][i], super_point_group["translations"][i], dic_zahyou)
        return jun
    
    rot_permutations = generate_super_perm(dic_zahyou, super_point_group)
    print(rot_permutations, "rotperm")
    print(trans_tikans, "trans_tikans")
    goal_perms = generate_abs_permuatation(rot_permutations, trans_tikans)
    
    return goal_perms

In [15]:
o_super_sublattice = generate_superlattice.get_o_superlattice(rutile, reduced_HNF[0], 3)
unko = shin_get_permutation(o_super_sublattice, parent_sym,  reduced_HNF[0])

{0: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11}, 1: {0: 1, 1: 0, 2: 11, 3: 10, 4: 9, 5: 8, 6: 7, 7: 6, 8: 5, 9: 4, 10: 3, 11: 2}, 2: {0: 2, 1: 3, 4: 6, 5: 7, 6: 9, 7: 8}, 3: {0: 3, 1: 2, 6: 8, 7: 9, 8: 7, 9: 6}, 4: {0: 1, 1: 0, 2: 3, 3: 2, 4: 5, 5: 4, 6: 7, 7: 6, 8: 9, 9: 8, 10: 11, 11: 10}, 5: {0: 0, 1: 1, 2: 10, 3: 11, 4: 8, 5: 9, 6: 6, 7: 7, 8: 4, 9: 5, 10: 2, 11: 3}, 6: {0: 3, 1: 2, 4: 7, 5: 6, 6: 8, 7: 9}, 7: {0: 2, 1: 3, 6: 9, 7: 8, 8: 6, 9: 7}, 8: {0: 3, 1: 2, 6: 9, 7: 8, 8: 7, 9: 6}, 9: {0: 2, 1: 3, 4: 6, 5: 7, 6: 8, 7: 9}, 10: {0: 1, 1: 0, 2: 10, 3: 11, 4: 9, 5: 8, 6: 6, 7: 7, 8: 5, 9: 4, 10: 2, 11: 3}, 11: {0: 0, 1: 1, 2: 3, 3: 2, 4: 4, 5: 5, 6: 7, 7: 6, 8: 8, 9: 9, 10: 11, 11: 10}, 12: {0: 2, 1: 3, 6: 8, 7: 9, 8: 6, 9: 7}, 13: {0: 3, 1: 2, 4: 7, 5: 6, 6: 9, 7: 8}, 14: {0: 0, 1: 1, 2: 11, 3: 10, 4: 8, 5: 9, 6: 7, 7: 6, 8: 4, 9: 5, 10: 3, 11: 2}, 15: {0: 1, 1: 0, 2: 2, 3: 3, 4: 5, 5: 4, 6: 6, 7: 7, 8: 9, 9: 8, 10: 10, 11: 11}} rotperm
{0: {0: 0

KeyError: 2

In [16]:
rot_tikan

NameError: name 'rot_tikan' is not defined